In [ ]:
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam

In [ ]:
# Завантаження даних
max_features = 5000  # Кількість найпоширеніших слів для токенізації
maxlen = 500  # Максимальна довжина кожної рецензії

# Завантаження тренувальних та тестових даних
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Паддінг для забезпечення однакової довжини
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
# Функція для створення та компіляції моделей
def create_model(model_type='RNN', units=128, dropout_rate=0.2):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
    
    if model_type == 'RNN':
        model.add(SimpleRNN(units, dropout=dropout_rate))
    elif model_type == 'LSTM':
        model.add(LSTM(units, dropout=dropout_rate))
    elif model_type == 'Bidirectional':
        model.add(Bidirectional(LSTM(units, dropout=dropout_rate)))
    elif model_type == 'Deep LSTM':
        model.add(LSTM(units, return_sequences=True, dropout=dropout_rate))
        model.add(LSTM(units, dropout=dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [ ]:
# Створення моделей
models = {
    'RNN': create_model('RNN'),
    'LSTM': create_model('LSTM'),
    'Bidirectional': create_model('Bidirectional'),
    'Deep LSTM': create_model('Deep LSTM')
}

In [ ]:
# Навчання моделей
history = {}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    history[model_name] = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

In [ ]:
# Побудова графіків
plt.figure(figsize=(12, 6))

# Графік залежності accuracy від епохи
plt.subplot(1, 2, 1)
for model_name, hist in history.items():
    plt.plot(hist.history['accuracy'], label=f'{model_name} (train)')
    plt.plot(hist.history['val_accuracy'], label=f'{model_name} (val)')
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Графік залежності loss від епохи
plt.subplot(1, 2, 2)
for model_name, hist in history.items():
    plt.plot(hist.history['loss'], label=f'{model_name} (train)')
    plt.plot(hist.history['val_loss'], label=f'{model_name} (val)')
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Висновок
LSTM показує найкращі результати на тренувальних даних із точністю 93.15%, також має найвищу точність на валідаційних даних (87.00%), 
що вказує на те, що ця модель найкраще узагальнює на нових даних. Також має найнижчі втрати на тренувальних даних (0.1811), що говорить 
про хорошу якість навчання.
    
Deep LSTM демонструє найгіршу валідаційну точність на останній епосі (68.40%), до чого можливо призводить перенавчання. 
Хоча точність на тренувальних даних висока (91.37%), але на валідаційних даних модель не працює добре.
Вона може бути занадто складною для цієї задачі.

Deep LSTM має високі коливання у втраті на валідаційних даних, що говоритьпро нестабільність в моделі.
Bidirectional LSTM показує добрі результати, але трохи гірші за LSTM.

Модель RNN є найменш ефективною серед цих чотирьох архітектур.